### Helper Classes

First we get all of our helper modules. The prepare_EMG module will prepare the EMG data for phoneme recognition. The prepare_outputs module will prepare our target labels and align them with our EMG data. The module 'prepare_data' will help us read data from CSV into a dataframe. Finally, 'vis' will help visualize EMG data in both time and frequency domains. 

In [3]:
%load_ext autoreload
%autoreload 2

import prepare_EMG, prepare_outputs, prepare_data, vis
# autodetector = Output_Prep.detector
EMG_Prep = prepare_EMG.EMG_preparer(window_size=30.0)
# Output_Prep = prepare_outputs.output_preparer(subvocal_detector = autodetector, window_size=30.0)
Output_Prep = prepare_outputs.output_preparer(window_size=30.0)

Data_Prep = prepare_data.data_preparer()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Score: 0.691699604743


### Labeling the Data

First, we need to visualize a few EMG voltage graphs to find some sections that most likely contain no subvocalization. Then, we'll need to find some regions that almost certainly do. These two classes of EMG readouts will serve to train an identifier to help us automatically label EMG windows with phonemes. The model used here will most likely be an SVC, inside "prepare_outputs". It will process each EMG window in order, and when it finds one that most likely contains subvocalization, it applies the next phoneme as that window's label. 

In [25]:
data_1 = Data_Prep.load('Sat Mar  4 00:44:23 2017')


In [21]:
a = 4/2
b = 4//2

a,b

(2.0, 2)

In [24]:
phoneme_list = Output_Prep.transform('Well Hello There!')
phoneme_list_2 = Output_Prep.transform("What's for dinner?")

print (phoneme_list, phoneme_list_2)



    manner      place   height   vowel
    silent     silent   silent  silent
AA   vowel       back      low     yes
AE   vowel  mid-front      low     yes
AH   vowel        mid      mid     yes
AO   vowel       back  mid-low     yes
W
EH
L
HH
AH
L
OW
DH
EH
R
              manner      place     height vowel
W        approximant       back  very-high    no
EH             vowel  mid-front        mid   yes
L        approximant    lateral  very high    no
HH         aspirated     uknown        max    no
AH             vowel        mid        mid   yes
L        approximant    lateral  very high    no
OW             vowel       back        mid   yes
DH  voiced-fricative     dental        max    no
EH             vowel  mid-front        mid   yes
R        approximant  retroflex    mid-low    no
    manner      place   height   vowel
    silent     silent   silent  silent
AA   vowel       back      low     yes
AE   vowel  mid-front      low     yes
AH   vowel        mid      mid     yes
AO   v

### Our Phonemes

We use a counter to see what phonemes nltk actually has in store for us, then we sort and display them in order. We can then better fill-in articulatory features from the 50 or so the rasipuram paper has. The nltk docs say there's only 39, but clearly there are a bit more here. We'll try to fill in articulatory features for all of the phonemes here, duplicating ones where we don't have unique AF's from the paper. We need to specify AF's for all of these because these are the phonemes we'll be classifying. 

In [14]:
from collections import Counter
phonemes = Counter()
values = Output_Prep.arpabet.values()
for list_1 in values:
    for list_2 in list_1:
        if len(list_2) == 1:
            phonemes.update(list_2)
        else:
            for item in list_2:
                phonemes.update([str(item)])

In [15]:
keys = list(phonemes.keys()) 
keys.sort()
keys

[autoreload of prepare_outputs failed: Traceback (most recent call last):
  File "/home/brian/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "/home/brian/Documents/Projects/MLND/p5/MLND-Subvocal/prepare_outputs.py", line 108
    vector =
           ^
SyntaxError: invalid syntax
]


['AA0',
 'AA1',
 'AA2',
 'AE0',
 'AE1',
 'AE2',
 'AH0',
 'AH1',
 'AH2',
 'AO0',
 'AO1',
 'AO2',
 'AW0',
 'AW1',
 'AW2',
 'AY0',
 'AY1',
 'AY2',
 'B',
 'CH',
 'D',
 'DH',
 'EH0',
 'EH1',
 'EH2',
 'ER0',
 'ER1',
 'ER2',
 'EY0',
 'EY1',
 'EY2',
 'F',
 'G',
 'HH',
 'IH0',
 'IH1',
 'IH2',
 'IY0',
 'IY1',
 'IY2',
 'JH',
 'K',
 'L',
 'M',
 'N',
 'NG',
 'OW0',
 'OW1',
 'OW2',
 'OY0',
 'OY1',
 'OY2',
 'P',
 'R',
 'S',
 'SH',
 'T',
 'TH',
 'UH0',
 'UH1',
 'UH2',
 'UW',
 'UW0',
 'UW1',
 'UW2',
 'V',
 'W',
 'Y',
 'Z',
 'ZH']

In [95]:
import pandas
%autoreload 2

labels_frame = pandas.read_csv('austen_subvocal.csv')
labels_frame
# print(labels_frame.iloc[0][0])
trans_labels = Output_Prep.transform(labels_frame.iloc[0][0])
data_1_proc = EMG_Prep.process(data_1)
print("This is trans_labels:",trans_labels)
# print('this is data_1_proc:',data_1_proc)
aligned_labels = Output_Prep.zip(data_1_proc, trans_labels, repeat=3)
print('this is aligned_labels:', aligned_labels)

This is trans_labels:               manner      place     height vowel
DH  voiced-fricative     dental        max    no
AH             vowel        mid        mid   yes
F          fricative     labial        max    no
AE             vowel  mid-front        low   yes
M              nasal     labial        max    no
AH             vowel        mid        mid   yes
L        approximant    lateral  very high    no
IY             vowel      front  very high   yes
AH             vowel        mid        mid   yes
V   voiced-fricative     labial        max    no
D        voiced-stop   alveolar        max    no
AE             vowel  mid-front        low   yes
SH         fricative      front        max    no
W        approximant       back  very-high    no
UH             vowel   mid-back       high   yes
D        voiced-stop   alveolar        max    no
HH         aspirated     uknown        max    no
AE             vowel  mid-front        low   yes
D        voiced-stop   alveolar        max    n